In [5]:
import ee, geemap, os, datetime
ee.Initialize()

year = 2015, 2015
doy = '-01-01', '-12-31'

start_date = str(year[0]) + doy[0]
end_date = str(year[1]) + doy[1]

jx_bound = ee.FeatureCollection('users/yehuigeo/jx_bound')

In [2]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2).divide(10000)

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0)).divide(10000)
    return image.updateMask(mask)

In [3]:
def f_EVI(img):
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select('B4'),
          'RED': img.select('B3'),
          'BLUE':img.select('B1')})
    return EVI
def f_GCVI(img):
    GCVI = img.expression(
        '(NIR / Green - 1)', {
          'NIR': img.select('B4'),
          'Green': img.select('B2')})
    return GCVI
def addVIs(img):
    NDWI = img.normalizedDifference(['B3','B5']).rename('NDWI')
    LSWI = img.normalizedDifference(['B5','B6']).rename('LSWI')
    EVI = f_EVI(img)
    GCVI = f_GCVI(img)
    return img.addBands(NDVI).addBands(NDWI).addBands(LSWI).addBands(EVI).addBands(GCVI)

In [6]:
# Landsat 3 MSS Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1978-03-05 - 1983-03-31
# Landsat 5+7 scenes
# Dataset Availability
# 1984-03-01 - 2008-07-29
# get landsat 5
collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
# print('Landsat5')
print(collection5.size().getInfo())
# get landsat 7
collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])

print(collection7.size().getInfo())

0
191


In [7]:
# get landsat 8
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(maskL8sr).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']) 
print(collection8.size().getInfo())
# get sentinel-2A
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate(start_date, end_date).map(maskS2clouds).filterBounds(jx_bound)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) 
print(sentinel2.size().getInfo())

232
0


In [8]:
Map = geemap.Map()
Map
# Map.addLayer(collection5.first(),{'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3})
# Map.addLayer(collection7.first(),{'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3})
# Map.addLayer(collection8.first(),{'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3})
# Map.addLayer(sentinel2.first(),  {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3})

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [20]:
Map.addLayer(sentinel2.median().clip(jx_bound),{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})

In [13]:
# print(collection7.median())


ee.Image({
  "type": "Invocation",
  "arguments": {
    "collection": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "collection": {
              "type": "Invocation",
              "arguments": {
                "collection": {
                  "type": "Invocation",
                  "arguments": {
                    "collection": {
                      "type": "Invocation",
                      "arguments": {
                        "id": "LANDSAT/LE07/C01/T1_SR"
                      },
                      "functionName": "ImageCollection.load"
                    },
                    "filter": {
                      "type": "Invocation",
                      "arguments": {
                        "leftField": ".all",
                        "rightValue": {
                          "type": "Invocation",
                          "arguments": {
                            "geo